  
<td>
    <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>

----

# Model Diagnostics - Custom Metrics


* The main idea behind custom metrics is that by enabling users to provide specific metrics, that closely align with company objectives, users will be able to iterate faster and more track model quality over time with less noise.

* For example, a self driving car dashcam might track the precision, recall, and iou of their model. However, nearby objects, such as people, might matter much more than far away objects. This important aspect of model quality is lost to the noise of these broad metrics. 
* Continuing with this example, the organization might want to report safety in terms of performance of the model on nearby objects. They maybe also want to know how much marginal value they are getting out of each training example. Both of these use cases rely on the metric specifically tracking the company's target objective.


<b>Topics Covered</b>
* Custom metrics basics
* Complete diagnostics demo using custom metrics



## Environment Setup

Install dependencies

In [1]:
!pip install -q "labelbox[data]" \
             scikit-image \
             tensorflow

zsh:1: command not found: pip


In [2]:
# Run these if running in a colab notebook
COLAB = "google.colab" in str(get_ipython())

if COLAB:
    !git clone https://github.com/Labelbox/labelbox-python.git
    !cd labelbox-python
    !mv labelbox-python/examples/model_assisted_labeling/*.py .

Import libraries

In [32]:
import sys
sys.path.append('../model_assisted_labeling')

import uuid
import numpy as np
from skimage import measure
import requests
from tqdm import notebook
import requests
import csv
import os

from labelbox.schema.ontology import OntologyBuilder, Tool
from labelbox.data.metrics.processing import get_label_pairs
from labelbox import Client, LabelingFrontend, MALPredictionImport
from labelbox.data.metrics.iou import data_row_miou, feature_miou_metric
from labelbox.data.serialization import NDJsonConverter
from labelbox.data.annotation_types import (
    ScalarMetric, 
    LabelList, 
    Label, 
    ImageData, 
    MaskData,
    Mask, 
    Polygon,
    Point, 
    Rectangle, 
    ObjectAnnotation,
    ClassificationAnnotation,
    ClassificationAnswer,
    Radio
)

try:
    from image_model import predict, load_model
except ModuleNotFoundError: 
    # !git clone https://github.com/Labelbox/labelbox-python.git
    # !cd labelbox-python && git checkout mea-dev
    # !mv labelbox-python/examples/model_assisted_labeling/*.py .
    raise Exception("You will need to run from the labelbox-python git repo")

## Custom Metrics
* Users can provide metrics to provide metric information at different levels of granularity.
    * Users can provide metrics for 
        1. data rows
        2. features
        3. subclasses
    * Additionally, metrics can be given custom names to best describe what they are measuring.
    
* Limits and Behavior:
    * At a data row cannot have more than 20 metrics
    * Metrics are upserted, so if a metric already exists, its value will be replaced
    * Metrics can have values in the range [0,100000]
* Currently `ScalarMetric`s and `ConfusionMatrixMetric`s are supported. 

### ScalarMetric
    - A `ScalarMetric` is a metric with just a single scalar value.

In [4]:
from labelbox.data.annotation_types import (
        ScalarMetric, 
    ScalarMetricAggregation, 
    ConfusionMatrixMetric
)

In [5]:
data_row_metric = ScalarMetric(
    metric_name = "iou",
    value = 0.5
)

feature_metric = ScalarMetric(
    metric_name = "iou",
    feature_name = "cat",
    value = 0.5
)

subclass_metric = ScalarMetric(
    metric_name = "iou",
    feature_name = "cat",
    subclass_name = "organge",
    value = 0.5
)


### ConfusionMatrixMetric
- A `ConfusionMatrixMetric` contains 4 numbers [True postivie, False Postive, True Negative, False Negateive]
- Confidence is also supported a key value pairs, where the score is the key and the value is the metric value.
- In the user interface, these metrics are used to derive precision,recall, and f1 scores. The reason these are not directly uploaded is that the raw data allows us to do processing on the front end.


In [6]:

data_row_metric = ConfusionMatrixMetric(
    metric_name = "confusion_matrix_50pct_iou",
    feature_name = "cat",  
    subclass_name = "organge",    
    value = [1,0,1,0]
)


feature_metric = ConfusionMatrixMetric(
    metric_name = "confusion_matrix_50pct_iou",
    feature_name = "cat",  
    subclass_name = "organge",    
    value = [1,0,1,0]
)

subclass_metric = ConfusionMatrixMetric(
    metric_name = "confusion_matrix_50pct_iou",
    feature_name = "cat",  
    subclass_name = "organge",    
    value = [1,0,1,0]
)


### Confidence
* Users can provide confidence scores along with metrics
* This enables them to explore their model performance without necessarily knowing the optimal thresholds for each class.
* Users can filter on confidence and value in the UI to perform powerful queries.
* The keys represent a confidence score (must be between 0 and 1) and the values represent either a scalar metric or for confusion matrix metrics [TP,FP,TN,FN]

In [7]:

confusion_matrix_metric_with_confidence = ConfusionMatrixMetric(
    metric_name = "confusion_matrix_50pct_iou",
    feature_name = "cat",  
    subclass_name = "organge",    
    value = {0.1 : [1,0,1,0], 0.3 : [1,0,1,0], 0.5 : [1,0,1,0], 0.7 : [1,0,1,0], 0.9 : [1,0,1,0]}
)

scalar_metric_with_confidence = ScalarMetric(
    metric_name = "iou",
    value = {0.1 : 0.2, 0.3 : 0.25, 0.5 : 0.3, 0.7 : 0.4, 0.9: 0.3}
)



### Aggregations
* This is an optional field on the `ScalarMetric` object (by default it uses Arithmetic Mean).
* Aggregations occur in two cases:
    1. When a user provides a feature or subclass level metric, Labelbox automatically aggregates all metrics with the same parent to create a value for that parent.
        * E.g. A user provides cat and dog iou. The data row level metric for iou is the average of both of those.
        * The exception to this is when the data row level iou is explicitly set, then the aggregation will not take effect (on a per data row basis). 
    2. When users create slices or want aggregate statistics on their models, the selected aggregation is applied.

In [8]:
"""
If the following metrics are uploaded then
in the web app, users will see:
true positives dog = 4
true positives cat = 3
true positives = 7
"""

feature_metric = ScalarMetric(
    metric_name = "true_positives",
    feature_name = "cat",
    value = 3,
    aggregation = ScalarMetricAggregation.SUM
)

feature_metric = ScalarMetric(
    metric_name = "true_positives",
    feature_name = "dog",
    value = 4,
    aggregation = ScalarMetricAggregation.SUM
)


### Built-in Metrics:
* The SDK Provides a set of default metrics that make metrics easy to use.
1. `confusion_matrix_metric()`
    * Computes a single confusion matrix metric for all the predictions and labels provided. 
2. `miou_metric()`
    * Computes a single iou score for all predictions and labels provided    
3. `feature_confusion_matrix_metric()`
    * Computes the iou score for each of the classes found in the predictions and labels
4. `feature_miou_metric()`
     * Computes a confusion matrix metric for each of the classes found in the predictions and labels

In [20]:
from labelbox.data.metrics import feature_miou_metric, miou_metric, confusion_matrix_metric, feature_confusion_matrix_metric
predictions = [
    ObjectAnnotation(
        name="cat",
        value=Rectangle(start=Point(x=0, y=0),
                        end=Point(x=10, y=10))
    )
]
        
ground_truths = [
    ObjectAnnotation(
        name="cat",
        value=Rectangle(start=Point(x=0, y=0),
                        end=Point(x=8, y=8)))
]

In [21]:
print(feature_miou_metric(ground_truths, predictions))
print(miou_metric(ground_truths, predictions))
print(confusion_matrix_metric(ground_truths, predictions))
print(feature_confusion_matrix_metric(ground_truths, predictions))

[ScalarMetric(value=0.64, feature_name='cat', subclass_name=None, extra={}, metric_name='iou', aggregation=<ScalarMetricAggregation.ARITHMETIC_MEAN: 'ARITHMETIC_MEAN'>)]
[ScalarMetric(value=0.64, feature_name=None, subclass_name=None, extra={}, metric_name='iou', aggregation=<ScalarMetricAggregation.ARITHMETIC_MEAN: 'ARITHMETIC_MEAN'>)]
[ConfusionMatrixMetric(value=(1, 0, 0, 0), feature_name=None, subclass_name=None, extra={}, metric_name='confusion_matrix_50pct_iou', aggregation=<ConfusionMatrixAggregation.CONFUSION_MATRIX: 'CONFUSION_MATRIX'>)]
[ConfusionMatrixMetric(value=(1, 0, 0, 0), feature_name='cat', subclass_name=None, extra={}, metric_name='confusion_matrix_50pct_iou', aggregation=<ConfusionMatrixAggregation.CONFUSION_MATRIX: 'CONFUSION_MATRIX'>)]


In [24]:
# Adjust iou for iou calcuations.
# Set it higher than 0.64 and we get a false postive and a false negative for the other ground truth object.
print(feature_confusion_matrix_metric(ground_truths, predictions, iou = 0.9))

[ConfusionMatrixMetric(value=(0, 1, 0, 1), feature_name='cat', subclass_name=None, extra={}, metric_name='confusion_matrix_90pct_iou', aggregation=<ConfusionMatrixAggregation.CONFUSION_MATRIX: 'CONFUSION_MATRIX'>)]


In [41]:
# subclasses are included by default
predictions = [
    ObjectAnnotation(
        name="cat",
        value=Rectangle(start=Point(x=0, y=0),
                        end=Point(x=10, y=10)),
    classifications = [
        ClassificationAnnotation(
        name="height", value=Radio(answer=ClassificationAnswer(name="tall")))
    
    ])
]

ground_truths = [
    ObjectAnnotation(
        name="cat",
        value=Rectangle(start=Point(x=0, y=0),
                        end=Point(x=10, y=10)),
    classifications = [
        ClassificationAnnotation(
        name="height", value=Radio(answer=ClassificationAnswer(name="short")))
    
    ])
]



In [40]:
print("Subclasses:", feature_confusion_matrix_metric(ground_truths, predictions)[0].value)
print("Excluding Subclasses:", feature_confusion_matrix_metric(ground_truths, predictions, include_subclasses = False)[0].value)

Subclasses: (0, 1, 0, 1)
Excluding Subclasses: (1, 0, 0, 0)


## Complete Example
* Custom metrics are uploaded exactly the same way that iou was previously uploaded.
* A metric must be added to a `Label` to create an association between a data row and the metric

In [42]:
PROJECT_NAME = "Diagnostics Demo Custom Metrics"
MODEL_NAME = "MSCOCO-Mapillary-Custom-Metrics"
MODEL_VERSION = "0.0.0"

In [43]:
client = Client(api_key=API_KEY)
load_model() # initialize Tensorflow Model

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from gs://cloud-tpu-checkpoints/mask-rcnn/1555659850/variables/variables


2021-09-13 08:27:06.891557: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:196] None of the MLIR optimization passes are enabled (registered 0 passes)


In [44]:
# Configure for whatever combination of tools and class names that you would like.
class_mappings = {
    1: {"name": 'person', "kind": Tool.Type.POLYGON},
    2: {"name": 'bicycle', "kind": Tool.Type.SEGMENTATION, 'color' : 64},
    3: {"name": 'car', "kind": Tool.Type.BBOX},
    4: {"name": 'motorcycle', "kind": Tool.Type.BBOX},
    6: {"name": 'bus', "kind": Tool.Type.POLYGON},
    7: {"name": 'train', "kind": Tool.Type.POLYGON},
    8: {"name": 'truck', "kind": Tool.Type.POLYGON},
    10: {"name": 'traffic light', "kind": Tool.Type.POINT},
    11: {"name": 'fire hydrant', "kind": Tool.Type.BBOX},
    13: {"name": 'stop sign', "kind": Tool.Type.SEGMENTATION, 'color' : 255},
    14: {"name": 'parking meter', "kind": Tool.Type.POINT},
    28: {"name": 'umbrella', "kind": Tool.Type.SEGMENTATION, 'color' : 128},    
    31: {"name": 'handbag', "kind": Tool.Type.POINT},        
}

## Create Predictions
* Loop over data_rows, make predictions, and create ndjson

In [45]:
# --- setup dataset ---
# load mapillary sample
sample_csv_url = "https://raw.githubusercontent.com/Labelbox/labelbox-python/develop/examples/assets/mapillary_sample.csv"
with requests.get(sample_csv_url, stream=True) as r:
    image_data = [row.split(',') for row in (line.decode('utf-8') for line in r.iter_lines())]

In [46]:
predictions = LabelList()
for (image_url, external_id) in notebook.tqdm(image_data[:5]):
    image = ImageData(url = image_url, external_id = external_id)
    height, width = image.value.shape[:2]
    prediction = predict(np.array([image.im_bytes]), min_score=0.5, height=height, width = width)
    boxes, classes, seg_masks = prediction["boxes"], prediction["class_indices"], prediction["seg_masks"]
    annotations = []
    for box, class_idx, seg in zip(boxes, classes, seg_masks):
        if class_idx in class_mappings:
            class_info = class_mappings.get(class_idx)
            if class_info['kind'] == Tool.Type.POLYGON:
                contours = measure.find_contours(seg, 0.5)
                pts = contours[0].astype(np.int32)
                value = Polygon(points = [Point(x = x, y = y) for x,y in np.roll(pts, 1, axis=-1)])
            elif class_info['kind'] == Tool.Type.BBOX:
                value = Rectangle(start = Point(x = box[1], y = box[0]), end = Point(x=box[3], y=box[2]))
            elif class_info['kind'] == Tool.Type.POINT:
                value = Point(x=(box[1] + box[3]) / 2., y = (box[0] + box[2]) / 2.)
            elif class_info['kind'] == Tool.Type.SEGMENTATION:
                value = Mask(mask = MaskData.from_2D_arr(seg * class_info['color']), color = (class_info['color'],)* 3)
            else:
                raise ValueError(f"Unsupported kind found. {class_info['kind']}")
            annotations.append(ObjectAnnotation(name = class_info['name'], value = value))
    predictions.append(Label(data = image, annotations = annotations))

  0%|          | 0/5 [00:00<?, ?it/s]

## Setup a project

In [47]:
tools = []
for target in class_mappings.values():
     tools.append(Tool(tool=target['kind'], name=target["name"]))
ontology_builder = OntologyBuilder(tools=tools)

In [49]:
print(f"Setting up: {PROJECT_NAME}")

project = client.create_project(name=PROJECT_NAME)
editor = next(client.get_labeling_frontends(where=LabelingFrontend.name == "Editor"))
project.setup(editor, ontology_builder.asdict())

dataset = client.create_dataset(name="Mapillary Diagnostics Demo")
print(f"Dataset Created: {dataset.uid}")
project.datasets.connect(dataset)

Setting up: Diagnostics Demo Custom Metrics
Dataset Created: cktimhcy6c8mv0yba7onpdna9


## Prepare for upload
* Our local annotations need the following:
    1. signed url for segmentation masks
    2. data rows in labelbox
    3. feature schema ids

In [50]:
signer = lambda _bytes: client.upload_data(content=_bytes, sign=True)
predictions.add_url_to_masks(signer) \
         .add_url_to_data(signer) \
         .assign_feature_schema_ids(OntologyBuilder.from_project(project)) \
         .add_to_dataset(dataset, client.upload_data)

5it [00:00, 37052.16it/s]
5it [00:00, 46707.17it/s]
5it [00:00, 34267.19it/s]


## **Optional** - Create labels with [Model Assisted Labeling](https://docs.labelbox.com/en/core-concepts/model-assisted-labeling)

* Pre-label image so that we can quickly create ground truth
* Create ground truth data for Model Diagnostics
* Click on link below to label

In [53]:
RUN_MAL = True
if RUN_MAL:
    project.enable_model_assisted_labeling()
    # Convert from annotation types to import format
    ndjson_predictions = NDJsonConverter.serialize(predictions)
    upload_task = MALPredictionImport.create_from_objects(client, project.uid, f'mal-import-{uuid.uuid4()}',ndjson_predictions )
    upload_task.wait_until_done()
    print(upload_task.state , '\n')

[]


In [52]:
print(f"https://app.labelbox.com/go-label/{project.uid}")

https://app.labelbox.com/go-label/cktimhae82yuq0y6dbralbxkq


## Export Labels

We do not support `Skipped` labels and have a limit of **2000**

In [54]:
MAX_LABELS = 2000
labels = [l for idx, l in enumerate(project.label_generator()) if idx < MAX_LABELS]

## Setup Model & Model Run

In [56]:
lb_model = client.create_model(name = MODEL_NAME, ontology_id = project.ontology().uid)
lb_model_run = lb_model.create_model_run(MODEL_VERSION)

Select label ids to upload

In [57]:
lb_model_run.upsert_labels([label.uid for label in labels])

True

### Compute Metrics
* First get pairs of labels and predictions

In [58]:
pairs = get_label_pairs(labels, predictions, filter_mismatch = True)

* Create helper functions for our metrics
* All functions will accept ground truth and prediction annotations

In [59]:
from shapely.ops import cascaded_union

def nearby_cars_iou(ground_truths, predictions, area_threshold = 17000):
    """
    Metric to track the iou score for cars that are nearby (determined by pixel size).
    
    This might be useful to investigate why the model poorly when vehicles are nearby.
    Or this might just be a metric we care a lot about optimizing because our self driving car needs to 
     be aware of its immediate surroundings for safety reasons.
    """
    ground_truths = [gt for gt in ground_truths if gt.name == 'car']
    predictions   = [pred for pred in predictions if pred.name == 'car']
    ground_truths = cascaded_union([gt.value.shapely for gt in ground_truths if gt.value.shapely.area > area_threshold])
    predictions   = cascaded_union([pred.value.shapely for pred in predictions if pred.value.shapely.area > area_threshold])
    union = ground_truths.union(predictions).area
    # If there is no prediction or label then the score is undefined
    if union == 0:
        return []
    return [ScalarMetric(
            value = ground_truths.intersection(predictions).area / union,
            metric_name = "iou",
            feature_name = "car",
            subclass_name = "nearby" # Doesn't necessarily need to be a subclass in the ontology
        )] 



* Compute and sssign each metric to prediction label

In [60]:
for (ground_truth, prediction) in pairs.values():
    metrics = []
    metrics.extend(feature_miou_metric(ground_truth.annotations, prediction.annotations))
    metrics.extend(feature_confusion_matrix_metric(ground_truth.annotations, prediction.annotations))    
    metrics.extend(nearby_cars_iou(ground_truth.annotations, prediction.annotations))
    prediction.annotations.extend(metrics)

### Upload to Labelbox

In [61]:
upload_task = lb_model_run.add_predictions(f'diagnostics-import-{uuid.uuid4()}', NDJsonConverter.serialize(predictions))
upload_task.wait_until_done()
print(upload_task.state)

AnnotationImportState.FINISHED


### Open Model Run

In [62]:
for idx, annotation_group in enumerate(lb_model_run.annotation_groups()):
    if idx == 5:
        break
    print(annotation_group.url)

https://app.labelbox.com/models/9c4ca3f6-8fc0-0595-8841-9581434e9672/9c4ca3f7-8a53-089f-fcc0-f246531fb14c/AllDatarowsSlice/2b404c28-059e-4163-a5d0-d77e6f329ab5?view=carousel
https://app.labelbox.com/models/9c4ca3f6-8fc0-0595-8841-9581434e9672/9c4ca3f7-8a53-089f-fcc0-f246531fb14c/AllDatarowsSlice/533847fd-a3c7-48e6-886b-f9458497f403?view=carousel
https://app.labelbox.com/models/9c4ca3f6-8fc0-0595-8841-9581434e9672/9c4ca3f7-8a53-089f-fcc0-f246531fb14c/AllDatarowsSlice/9f4833a4-ea2c-4f24-9283-82a6ea758b2e?view=carousel
https://app.labelbox.com/models/9c4ca3f6-8fc0-0595-8841-9581434e9672/9c4ca3f7-8a53-089f-fcc0-f246531fb14c/AllDatarowsSlice/c1932683-d7c2-41c6-a186-a72569a4ebb0?view=carousel
https://app.labelbox.com/models/9c4ca3f6-8fc0-0595-8841-9581434e9672/9c4ca3f7-8a53-089f-fcc0-f246531fb14c/AllDatarowsSlice/e9ce5747-1921-49ed-b648-286a7858cfd9?view=carousel


In [76]:
for p in predictions:
    print([[x.value, x.feature_name] for x in p.annotations if not isinstance(x.value, Geometry)])

[[0.7599894332796007, 'car'], [(1, 0, 0, 0), 'car'], [0.7599894332796007, 'car']]
[[0.999990163223662, 'car'], [0.8, 'person'], [0.0, 'truck'], [(6, 0, 0, 0), 'car'], [(4, 1, 0, 0), 'person'], [(0, 1, 0, 0), 'truck'], [0.9999943560730941, 'car']]
[[0.0, 'truck'], [(0, 1, 0, 0), 'truck']]
[[0.6900109281465653, 'car'], [0.9090909090909091, 'person'], [(3, 1, 0, 0), 'car'], [(10, 1, 0, 0), 'person'], [0.8905576670555548, 'car']]
[[0.8707948102228716, 'car'], [(3, 0, 0, 0), 'car'], [0.8899172370104976, 'car']]


In [71]:
from labelbox.data.annotation_types import Geometry